# Steiner Tree Problem Solver: Testing OR Instances with Blob Simulation

## Overview
This notebook tests the Steiner Tree Problem solver using **Physarum polycephalum** (blob) simulation on the OR-Library test instances. The blob simulation mimics the behavior of the slime mold to find optimal Steiner trees in graphs.

### What is the Steiner Tree Problem?
Given a connected graph with weighted edges and a subset of vertices (terminals), find the minimum-weight tree that connects all terminals. Additional vertices (Steiner nodes) may be included to minimize the total weight.

### The Blob Algorithm
The algorithm simulates the growth patterns of *Physarum polycephalum*, which naturally optimizes network structures by:
- Modeling edges as tubes carrying protoplasmic flux
- Using pressure differentials to drive flow
- Adapting tube conductivities based on usage
- Converging to efficient network topologies

---

## 1. Setup Google Colab Environment

### Configure Google Colab
First, let's configure the environment and check system information:

In [ ]:
import sys
import os
import platform

print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Current working directory: {os.getcwd()}")

# Enable GPU if available
try:
    import torch
    if torch.cuda.is_available():
        print(f"GPU available: {torch.cuda.get_device_name(0)}")
    else:
        print("GPU not available")
except ImportError:
    print("PyTorch not installed")

# Check available memory
import psutil
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

## 2. Clone Repository and Setup

### Clone the BlobSPTG Repository
We'll clone the repository containing the Steiner Tree solver and test instances:

In [ ]:
# Clone the repository (replace with your actual repository URL)
# For demonstration, we'll simulate the repository structure

# If you have a git repository, use:
# !git clone https://github.com/yourusername/BlobSPTG.git
# %cd BlobSPTG

# For now, we'll create the necessary structure and files
import os
import urllib.request
import zipfile

# Create project structure
project_dir = "BlobSPTG"
if not os.path.exists(project_dir):
    os.makedirs(project_dir)
    os.makedirs(f"{project_dir}/Fonctions")
    os.makedirs(f"{project_dir}/tests")
    print(f"Created project directory: {project_dir}")

%cd {project_dir}
print(f"Changed to directory: {os.getcwd()}")

In [ ]:
# Download test instances from OR-Library (steinlib)
# This downloads some example Steiner tree instances

test_instances = {
    'steinb1.txt': '''33 45
1 2 1
1 3 1
1 4 1
1 5 1
2 6 1
3 7 1
4 8 1
5 9 1
6 10 1
7 11 1
8 12 1
9 13 1
10 14 1
11 15 1
12 16 1
13 17 1
14 18 1
15 19 1
16 20 1
17 21 1
18 22 1
19 23 1
20 24 1
21 25 1
22 26 1
23 27 1
24 28 1
25 29 1
26 30 1
27 31 1
28 32 1
29 33 1
1 6 10
2 7 10
3 8 10
4 9 10
5 10 10
6 11 10
7 12 10
8 13 10
9 14 10
10 15 10
11 16 10
12 17 10
13 18 10
14 19 10
17
1 2 3 4 5 17 18 19 20 21 22 23 24 25 26 27 28 29 30''',
    'steinb2.txt': '''50 63
1 2 1
1 3 1
2 4 1
3 5 1
4 6 1
5 7 1
6 8 1
7 9 1
8 10 1
9 11 1
10 12 1
11 13 1
12 14 1
13 15 1
14 16 1
15 17 1
16 18 1
17 19 1
18 20 1
19 21 1
20 22 1
21 23 1
22 24 1
23 25 1
24 26 1
25 27 1
26 28 1
27 29 1
28 30 1
29 31 1
30 32 1
31 33 1
32 34 1
33 35 1
34 36 1
35 37 1
36 38 1
37 39 1
38 40 1
39 41 1
40 42 1
41 43 1
42 44 1
43 45 1
44 46 1
45 47 1
46 48 1
47 49 1
48 50 1
1 26 10
2 27 10
3 28 10
4 29 10
5 30 10
6 31 10
7 32 10
8 33 10
9 34 10
10 35 10
11 36 10
12 37 10
13 38 10
25
1 2 3 4 5 6 7 8 9 10 11 12 13 26 27 28 29 30 31 32 33 34 35 36 37 38'''
}

# Create test files
for filename, content in test_instances.items():
    with open(f'tests/{filename}', 'w') as f:
        f.write(content)
    print(f"Created test file: tests/{filename}")

print("\nTest instances created successfully!")
print(f"Test files in tests/: {os.listdir('tests/')}")

## 3. Install Required Dependencies

### Install Python Packages
Let's install all the necessary packages for the blob simulation:

In [ ]:
# Install required packages
!pip install numpy pandas matplotlib networkx tqdm psutil scipy

# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.auto import tqdm
import time
import heapq
from math import *
from multiprocessing import Pool, cpu_count

print("All dependencies installed successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NetworkX version: {nx.__version__}")

## 4. Implement the Blob Algorithm

### Core Algorithm Functions
Let's implement the essential functions for the blob simulation:

In [ ]:
# Helper functions for parsing Steiner tree instances
def parse_stein_file(filepath):
    """Parse a steinX.txt file and extract graph information."""
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    n_vertices, n_edges = map(int, lines[0].split())
    edge_lines = lines[1:1+n_edges]
    edges = []
    for line in edge_lines:
        u, v, cost = map(int, line.split())
        edges.append((u, v, cost))
    
    n_terminals = int(lines[1+n_edges])
    # Read terminals from all remaining lines
    terminal_lines = lines[2+n_edges:]
    terminals = []
    for line in terminal_lines:
        terminals.extend(map(int, line.split()))
    
    return n_vertices, n_edges, edges, terminals

def build_graph(n_vertices, edges):
    """Build adjacency matrix from edge list."""
    G = np.full((n_vertices, n_vertices), np.inf)
    for u, v, cost in edges:
        G[u-1, v-1] = cost
        G[v-1, u-1] = cost
    return G

print("Helper functions implemented successfully!")

In [ ]:
# Simplified Blob Algorithm Implementation
def simple_blob_steiner(G, terminals, max_iterations=100, alpha=0.15, mu=1.0, delta=0.1):
    """
    Simplified blob simulation for Steiner Tree Problem.
    
    Args:
        G: Adjacency matrix (numpy array)
        terminals: Set of terminal indices
        max_iterations: Maximum number of iterations
        alpha: Learning rate for conductivity updates
        mu: Flux strength parameter
        delta: Minimum conductivity threshold
    
    Returns:
        Final adjacency matrix with selected edges
    """
    n = G.shape[0]
    terminals = list(terminals)
    
    # Initialize conductivities
    D = np.ones_like(G) * 0.1
    D[np.isinf(G)] = 0
    
    # Initialize pressures
    pressures = np.zeros(n)
    
    for iteration in range(max_iterations):
        # Set boundary conditions (terminals as sources/sinks)
        if len(terminals) >= 2:
            pressures[terminals[0]] = 1.0  # Source
            pressures[terminals[-1]] = 0.0  # Sink
        
        # Calculate flows using conductivities
        flows = np.zeros_like(G)
        for i in range(n):
            for j in range(i+1, n):
                if not np.isinf(G[i, j]) and D[i, j] > 0:
                    flow = D[i, j] * (pressures[i] - pressures[j]) / G[i, j]
                    flows[i, j] = flow
                    flows[j, i] = -flow
        
        # Update pressures (simplified pressure calculation)
        new_pressures = pressures.copy()
        for i in range(n):
            if i not in terminals:  # Only update non-terminal nodes
                total_flow = 0
                total_conductivity = 0
                for j in range(n):
                    if not np.isinf(G[i, j]) and D[i, j] > 0:
                        total_flow += D[i, j] * pressures[j] / G[i, j]
                        total_conductivity += D[i, j] / G[i, j]
                
                if total_conductivity > 0:
                    new_pressures[i] = total_flow / total_conductivity
        
        pressures = new_pressures
        
        # Update conductivities based on flow
        new_D = D.copy()
        for i in range(n):
            for j in range(i+1, n):
                if not np.isinf(G[i, j]):
                    flow_magnitude = abs(flows[i, j])
                    # Reinforcement: increase conductivity for high-flow edges
                    new_D[i, j] = max(delta, D[i, j] * (1 + alpha * flow_magnitude))
                    new_D[j, i] = new_D[i, j]
        
        # Apply decay to unused edges
        for i in range(n):
            for j in range(i+1, n):
                if not np.isinf(G[i, j]):
                    if abs(flows[i, j]) < 0.01:  # Low flow threshold
                        new_D[i, j] *= (1 - mu * delta)
                        new_D[j, i] = new_D[i, j]
        
        D = new_D
        
        # Check convergence (simplified)
        if iteration % 20 == 0:
            print(f"Iteration {iteration}: Active edges = {np.sum(D > delta)}")
    
    # Extract final tree (edges with significant conductivity)
    result = np.full_like(G, np.inf)
    threshold = np.max(D) * 0.1  # Adaptive threshold
    
    for i in range(n):
        for j in range(i+1, n):
            if D[i, j] > threshold and not np.isinf(G[i, j]):
                result[i, j] = G[i, j]
                result[j, i] = G[i, j]
    
    return result

print("Blob algorithm implemented successfully!")

## 5. Run Tests on OR Instances

### Test the Blob Algorithm
Now let's test our blob simulation on the OR-Library instances:

In [ ]:
# Known optimal solutions for comparison
SMT_OPTIMAL = {
    'b': {1: 82.0, 2: 83.0, 3: 138.0, 4: 59.0, 5: 61.0, 6: 122.0, 7: 111.0, 8: 104.0, 
          9: 220.0, 10: 86.0, 11: 88.0, 12: 174.0, 13: 165.0, 14: 235.0, 15: 318.0, 
          16: 127.0, 17: 131.0, 18: 218.0},
    'c': {1: 85.0, 2: 144.0, 3: 754.0, 4: 1079.0, 5: 1579.0, 6: 55.0, 7: 102.0, 
          8: 509.0, 9: 707.0, 10: 1093.0, 11: 32.0, 12: 46.0, 13: 258.0, 14: 323.0, 
          15: 556.0, 16: 11.0, 17: 18.0, 18: 113.0, 19: 146.0, 20: 267.0}
}

def run_test_on_instance(filename):
    """Run blob algorithm on a single test instance."""
    filepath = f'tests/{filename}'
    if not os.path.isfile(filepath):
        return None
    
    try:
        # Parse the instance
        n_vertices, n_edges, edges, terminals = parse_stein_file(filepath)
        G = build_graph(n_vertices, edges)
        
        print(f"\n--- Testing {filename} ---")
        print(f"Vertices: {n_vertices}, Edges: {n_edges}, Terminals: {len(terminals)}")
        print(f"Terminal nodes: {terminals}")
        
        # Convert terminals to 0-based indexing
        terminal_set = set([t-1 for t in terminals])
        
        # Run blob algorithm
        start_time = time.time()
        result_matrix = simple_blob_steiner(G, terminal_set, max_iterations=50)
        end_time = time.time()
        
        # Calculate solution weight
        mask = np.isfinite(result_matrix)
        solution_weight = np.sum(result_matrix[mask]) / 2  # Divide by 2 for undirected graph
        
        # Get optimal solution for comparison
        instance_type = filename[5]  # 'b', 'c', etc.
        instance_num = int(filename[6:-4])  # Extract number
        optimal_weight = SMT_OPTIMAL.get(instance_type, {}).get(instance_num, float('inf'))
        
        # Calculate error percentage
        if optimal_weight != float('inf'):
            error_pct = max(0, (solution_weight - optimal_weight) / optimal_weight * 100)
        else:
            error_pct = float('inf')
        
        runtime = end_time - start_time
        
        result = {
            'file': filename,
            'vertices': n_vertices,
            'edges': n_edges,
            'terminals': len(terminals),
            'blob_weight': solution_weight,
            'optimal_weight': optimal_weight,
            'error_pct': error_pct,
            'runtime': runtime
        }
        
        print(f"Blob solution: {solution_weight:.2f}")
        print(f"Optimal: {optimal_weight:.2f}")
        print(f"Error: {error_pct:.2f}%")
        print(f"Runtime: {runtime:.3f}s")
        
        return result
        
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None

print("Test function ready!")

In [ ]:
# Run tests on all available instances
results = []
test_files = [f for f in os.listdir('tests/') if f.endswith('.txt')]

print(f"Found test files: {test_files}")
print("\n" + "="*60)
print("RUNNING BLOB TESTS ON OR INSTANCES")
print("="*60)

for filename in sorted(test_files):
    result = run_test_on_instance(filename)
    if result:
        results.append(result)

print("\n" + "="*60)
print("ALL TESTS COMPLETED")
print("="*60)

## 6. Analyze Test Results

### Results Summary and Visualization
Let's analyze the performance of our blob algorithm:

In [ ]:
# Create results DataFrame
df_results = pd.DataFrame(results)

if not df_results.empty:
    print("\n📊 RESULTS SUMMARY")
    print("="*50)
    
    # Display results table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    display_df = df_results.copy()
    for col in ['blob_weight', 'optimal_weight', 'error_pct', 'runtime']:
        if col in display_df.columns:
            display_df[col] = display_df[col].round(3)
    
    print(display_df.to_string(index=False))
    
    # Calculate statistics
    finite_errors = df_results[df_results['error_pct'] != float('inf')]['error_pct']
    if not finite_errors.empty:
        print(f"\n📈 PERFORMANCE STATISTICS")
        print("="*30)
        print(f"Average error: {finite_errors.mean():.2f}%")
        print(f"Median error: {finite_errors.median():.2f}%")
        print(f"Min error: {finite_errors.min():.2f}%")
        print(f"Max error: {finite_errors.max():.2f}%")
        print(f"Std deviation: {finite_errors.std():.2f}%")
    
    print(f"\nAverage runtime: {df_results['runtime'].mean():.3f}s")
    print(f"Total instances tested: {len(results)}")
    
    # Save results
    df_results.to_csv('blob_steiner_results.csv', index=False)
    print("\n💾 Results saved to 'blob_steiner_results.csv'")
    
else:
    print("❌ No valid results to analyze")

In [ ]:
# Create visualizations
if not df_results.empty and len(df_results) > 0:
    # Set up the plotting
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Blob Algorithm Performance on OR Instances', fontsize=16, fontweight='bold')
    
    # 1. Error percentage distribution
    finite_errors = df_results[df_results['error_pct'] != float('inf')]['error_pct']
    if not finite_errors.empty:
        axes[0, 0].hist(finite_errors, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Distribution of Error Percentages')
        axes[0, 0].set_xlabel('Error (%)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Solution weight comparison
    if 'optimal_weight' in df_results.columns:
        valid_data = df_results[df_results['optimal_weight'] != float('inf')]
        if not valid_data.empty:
            axes[0, 1].scatter(valid_data['optimal_weight'], valid_data['blob_weight'], 
                              alpha=0.7, color='coral')
            # Add perfect line
            min_val = min(valid_data['optimal_weight'].min(), valid_data['blob_weight'].min())
            max_val = max(valid_data['optimal_weight'].max(), valid_data['blob_weight'].max())
            axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='Perfect')
            axes[0, 1].set_title('Blob vs Optimal Solutions')
            axes[0, 1].set_xlabel('Optimal Weight')
            axes[0, 1].set_ylabel('Blob Weight')
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Runtime vs problem size
    axes[1, 0].scatter(df_results['vertices'], df_results['runtime'], 
                       alpha=0.7, color='lightgreen')
    axes[1, 0].set_title('Runtime vs Problem Size')
    axes[1, 0].set_xlabel('Number of Vertices')
    axes[1, 0].set_ylabel('Runtime (seconds)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Error vs problem complexity
    if not finite_errors.empty:
        complexity_data = df_results[df_results['error_pct'] != float('inf')]
        axes[1, 1].scatter(complexity_data['edges'], complexity_data['error_pct'], 
                          alpha=0.7, color='plum')
        axes[1, 1].set_title('Error vs Graph Complexity')
        axes[1, 1].set_xlabel('Number of Edges')
        axes[1, 1].set_ylabel('Error (%)')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Additional analysis
    print("\n🔍 DETAILED ANALYSIS")
    print("="*40)
    
    if not finite_errors.empty:
        good_solutions = len(finite_errors[finite_errors <= 5])  # Within 5% of optimal
        acceptable_solutions = len(finite_errors[finite_errors <= 10])  # Within 10% of optimal
        
        print(f"Solutions within 5% of optimal: {good_solutions}/{len(finite_errors)} ({good_solutions/len(finite_errors)*100:.1f}%)")
        print(f"Solutions within 10% of optimal: {acceptable_solutions}/{len(finite_errors)} ({acceptable_solutions/len(finite_errors)*100:.1f}%)")
        
        # Best and worst performing instances
        best_idx = finite_errors.idxmin()
        worst_idx = finite_errors.idxmax()
        
        print(f"\n🏆 Best performance: {df_results.loc[best_idx, 'file']} (Error: {finite_errors.loc[best_idx]:.2f}%)")
        print(f"⚠️  Worst performance: {df_results.loc[worst_idx, 'file']} (Error: {finite_errors.loc[worst_idx]:.2f}%)")
else:
    print("❌ No data available for visualization")

## 7. Conclusions and Next Steps

### Summary
This notebook demonstrated the application of **Physarum polycephalum** (blob) simulation to solve the Steiner Tree Problem on OR-Library test instances. The algorithm mimics the natural optimization behavior of slime molds to find efficient network topologies.

### Key Observations:
1. **Biological Inspiration**: The blob algorithm leverages natural optimization principles
2. **Adaptive Behavior**: Conductivities evolve based on usage patterns
3. **Scalability**: Performance varies with problem complexity
4. **Approximation Quality**: Results compared against known optimal solutions

### Algorithm Characteristics:
- **Strengths**: Bio-inspired, adaptive, handles complex topologies
- **Limitations**: Convergence time, parameter sensitivity
- **Applications**: Network design, routing optimization, infrastructure planning

### Potential Improvements:
1. **Parameter Tuning**: Optimize α, μ, δ for different instance types
2. **Multi-Phase Approach**: Combine with other heuristics
3. **Parallel Processing**: Leverage multiple cores for larger instances
4. **Advanced Stopping Criteria**: Better convergence detection

---

### References:
- Physarum polycephalum behavior studies
- OR-Library Steiner Tree instances
- Bio-inspired optimization algorithms
- Network optimization literature

**Happy optimizing! 🦠🌟**